In [1]:
import os
os.chdir("../../")#与easymlops同级目录

### 拆分数据

In [2]:
import pandas as pd
data=pd.read_csv("./data/demo.csv")
data.head(5)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
x_train=data[:500]
x_test=data[500:]
y_train=x_train["Survived"]
y_test=x_test["Survived"]
del x_train["Survived"]
del x_test["Survived"]

### pipe构建

In [4]:
from easymlops import TablePipeLine
from easymlops.table.preprocessing import *
from easymlops.table.encoding import *
from easymlops.table.regression import *

In [5]:
table=TablePipeLine()
table.pipe(FixInput())\
  .pipe(FillNa())\
  .pipe(OneHotEncoding(cols=["Pclass", "Sex"], drop_col=False)) \
  .pipe(WOEEncoding(cols=["Ticket", "Embarked", "Cabin", "Sex", "Pclass"], y=y_train)) \
  .pipe(LabelEncoding(cols=["Name"]))\
  .pipe(LGBMRegression(y=y_train,objective="poisson"))

x_test_new=table.fit(x_train).transform(x_test)
x_test_new.head(5)

,pred
500,0.092996
501,0.716432
502,0.654662
503,0.309583
504,0.916929


In [6]:
#获取sabass特征重要性
table[-1].get_contrib(table.transform_single(x_test.to_dict("record")[0],run_to_layer=-2))

{'Sex': -0.5899834278029109,
 'Cabin': -0.13150175316983173,
 'Fare': -0.1301026457360956,
 'Age': 0.275645992578784,
 'PassengerId': -0.39285142163510006,
 'Pclass_2': 0.0868239954973954,
 'SibSp': 0.048242598427820915,
 'Parch': -0.01823350433998343,
 'Pclass': -0.01402473665988247,
 'Ticket': -0.0030424253099006976,
 'Embarked': -0.005601108385329127}

### 性能、一致性测试

In [7]:
table.auto_test(x_test[:10])


###################################################################
 1.一致性测试和性能测试:check_transform_function                      
###################################################################
(<class 'easymlops.table.preprocessing.core.FixInput'>) module check [transform] complete,speed:[0.3ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.preprocessing.onevar_operation.FillNa'>) module check [transform] complete,speed:[0.0ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.encoding.OneHotEncoding'>) module check [transform] complete,speed:[0.0ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.encoding.WOEEncoding'>) module check [transform] complete,speed:[0.0ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.encoding.LabelEncoding'>) module check [transform] complete,speed:[0.0ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.regression.LGBMRegression'>) module check [transform] complete,speed:[6.33ms]/it,cpu:[83%],memory:[0K]

#######################################

### 持久化测试

In [8]:
table.save("ml.pkl")

In [9]:
table=TablePipeLine()
table.pipe(FixInput())\
  .pipe(FillNa())\
  .pipe(OneHotEncoding(cols=["Pclass", "Sex"], drop_col=False)) \
  .pipe(WOEEncoding(cols=["Ticket", "Embarked", "Cabin", "Sex", "Pclass"])) \
  .pipe(LabelEncoding(cols=["Name"]))\
  .pipe(LGBMRegression(objective="poisson"))

table.load("ml.pkl")

In [10]:
table.auto_test(x_test[:10])


###################################################################
 1.一致性测试和性能测试:check_transform_function                      
###################################################################
(<class 'easymlops.table.preprocessing.core.FixInput'>) module check [transform] complete,speed:[0.0ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.preprocessing.onevar_operation.FillNa'>) module check [transform] complete,speed:[0.0ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.encoding.OneHotEncoding'>) module check [transform] complete,speed:[0.0ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.encoding.WOEEncoding'>) module check [transform] complete,speed:[0.0ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.encoding.LabelEncoding'>) module check [transform] complete,speed:[0.0ms]/it,cpu:[0%],memory:[0K]
(<class 'easymlops.table.regression.LGBMRegression'>) module check [transform] complete,speed:[1.56ms]/it,cpu:[0%],memory:[0K]

########################################

In [11]:
table.transform(x_test[:5])

,pred
500,0.092996
501,0.716432
502,0.654662
503,0.309583
504,0.916929


In [12]:
x_test_new.head(5)

,pred
500,0.092996
501,0.716432
502,0.654662
503,0.309583
504,0.916929
